<a href="https://colab.research.google.com/github/dropthejase/ml_training/blob/main/ml_from_scratch/decision_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

**PREPARE DATA TO TEST MODEL**

I prepped the train/test data first because I had to continually use it to test my model and debug it!

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

df = pd.DataFrame(iris.data, columns=['sepal_length','sepal_width','petal_length','petal_width'])
df['target'] = iris.target

df = df.sample(frac=1) # shuffle data

X = df.drop('target',axis=1)
y = df['target']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)



**MY VERSION**

**Fit function**
1. Find the best split using a function, and store the split and column that gives the best IG as attributes
2. Then split X_train and Y_train based on the best_split and best_col
3. For each of self.left and self.right, call TreeNode() object and fit recursively
4. Be aware of base cases:
    - If max IG is 0, it's a leaf - show mean prediction
    - If current depth == max_depth, give mean prediction
    - If only one sample in dataset, return label in that sample
    - If > 1 sample, but they have the same label, return that label

**Best split function**

For class in classes...:
1. Sort Xs for class in ascending order and sort Y correspondingly
2. Split X in the middle
3. Find IG from splitting at boundary and store
4. Compare IGs amongst the classes
5. Store the best class/column, split, and IG which gives the max IG

**Predict function**
1. For each X_test row, pull best_col of X
2. Compare this value to best_split
3. If < best_split and self.left not empty, predict on self.left. Likewise if >= best_split. Otherwise if the self.left or self.right nodes are empty, give the self.prediction
4. If no self.col or self.split, then just give the prediction

In [ ]:
# Sort Xs for current column in ascending order, sort Y in corresponding way
# Find boundary points where Y changes from one value to another

def entropy(y):
  N = len(y)
  s1 = (y == 1).sum()
  if 0 == s1 or N == s1:
    return 0
  
  p1 = float(s1) / N
  p0 = 1 - p1

  return -p0*np.log2(p0) - p1*np.log2(p1)

class TreeNode:
  def __init__(self, depth=0, max_depth=2):
    self.depth = depth
    self.max_depth = max_depth
    if max_depth is not None and depth > max_depth:
      raise Exception("self_depth is larger than max_depth")

  def fit(self, X, Y):
    # if there's only 1 label in Y, so we can't split so we just predict the label
    if len(Y) == 1 or len(set(Y)) == 1:
      self.col = None
      self.split = None
      self.left = None
      self.right = None
      self.prediction = Y[0]
    
    # else give the best D, split, IG
    else:
      split = self.find_split(X, Y)
      best_col = split[0] # i.e. best D
      best_split = split[1]
      max_ig = split[2]

      # if max_ig == 0, then it's a leaf node
      if max_ig == 0:
        self.col = None
        self.split = None
        self.left = None
        self.right = None
        self.prediction = np.round(np.mean(Y))

      # else create the split
      else:
        self.col = best_col
        self.split = best_split

        # if max_depth reached...no more split, we have a prediction for the left and right sides
        if self.depth == self.max_depth:
          self.prediction = [
              np.round(Y[X[:, self.col] < self.split].mean()), # left
              np.round(Y[X[:, self.col] >= self.split].mean()), # right
          ]
      
        # if not max_depth then we need to do a recurssion and slice X, Y according to best_split and best_col
        else:
          self.left = TreeNode(depth=self.depth+1,max_depth=self.max_depth)
          Xleft = X[X[:, self.col] < self.split]
          Yleft = Y[X[:, self.col] < self.split]
          self.left.fit(Xleft, Yleft)

          self.right = TreeNode(depth=self.depth+1,max_depth=self.max_depth)
          Xright = X[X[:, self.col] >= self.split]
          Yright = Y[X[:, self.col] >= self.split]
          self.right.fit(Xright, Yright)         

  def find_split(self, X, Y):
    
    N, D = X.shape
    mid = int(N/2)-1

    # for each feature, sort the X, sort the Ys, find the IGs, get max IG, then return the optimal split
    igs = [] # dimension, split, IG
    
    for _ in range(D):
      temp_list = []
      temp_list.append(_)

      # sort the Xs and Ys for a given feature D
      X_sorted = X[:,_]
      sorted_idx = np.argsort(X_sorted)
      X_sorted = X_sorted[sorted_idx]
      Y_sorted = Y[sorted_idx]

      # take the midpoint of X values and split
      split = (X_sorted[mid] + X_sorted[mid+1]) / 2
      temp_list.append(split)

      ig = self.information_gain(X_sorted, Y_sorted, split)
      temp_list.append(ig)
      igs.append(temp_list)
    
    #print(igs)

    max_ig = max(igs, key=lambda x:x[2]) # get max IG in [D, split, IG] list
    return max_ig

  def information_gain(self, X, Y, split):
    N = len(Y)
    y0 = Y[X < split]
    y1 = Y[X >= split]

    w0 = len(y0) / N
    w1 = 1-w0

    return 1 - w0*entropy(y0) - w1*entropy(y1)

  def predict_one(self, X):
    if self.col is not None and self.split is not None:
      feature = X[self.col]
      if feature < self.split: # if less than split, go left
        if self.left: # if self.left is not None, recursion otherwise give left prediction
          p = self.left.predict_one(X)
        else:
          p = self.prediction[0]
  
      else:
        if self.right: # if self.right is not None, recursion otherwise give right prediction
          p = self.right.predict_one(X)
        else:
          p = self.prediction[1]
        
    else: # corresponds to having only 1 prediction
      p = self.prediction
      
    return p
  
  def predict(self, X):
    N = len(X)
    P = np.zeros(N)
    for i in range(N):
      P[i] = self.predict_one(X[i])
    return P

class DecisionTree:
  def __init__(self, max_depth=None):
    self.max_depth = max_depth

  def fit(self, X, Y):
    self.root = TreeNode(max_depth = self.max_depth)
    self.root.fit(X, Y)

  def predict(self, X):
    return self.root.predict(X)

  def score(self, X, Y):
    P = self.predict(X)
    return np.mean (P==Y)

In [ ]:
DT = DecisionTree()
DT.fit(X_train, y_train)
DT.predict(X_test)
DT.score(X_test, y_test)

0.92